# Chapter 10. Data Aggregation and Group Operations

## 10.1 GroupBy Mechanics

Hadley Wickham, an author of many popular packages for the R programming language, coined the term split-apply-combine for describing group operations. 
![Split Apply Combine](images/split-apply-combine.png)

In [1]:
import pandas as pd
import numpy as np

%config Completer.use_jedi = False

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                  'key2' : ['one', 'two', 'one', 'two', 'one'],
                  'data1' : [1, 2, 3, 4, 5],
                  'data2' : [10, 20, 30, 40, 50]})

df

,key1,key2,data1,data2
0,a,one,1,10
1,a,two,2,20
2,b,one,3,30
3,b,two,4,40
4,a,one,5,50


In [3]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [4]:
# the name of the index is 'key1'
grouped.mean()

key1
a    2.666667
b    3.500000
Name: data1, dtype: float64

In [5]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     3
      two     2
b     one     3
      two     4
Name: data1, dtype: int64

In [6]:
means.unstack()

key2,one,two
key1,,
a,3,2
b,3,4


In [7]:
# You can use arrays from outside to group data
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    2.0
            2006    3.0
Ohio        2005    2.5
            2006    5.0
Name: data1, dtype: float64

In [8]:
# non numerical data 'key2' is hidden (nuisance column)
df.groupby('key1').mean()

,data1,data2
key1,,
a,2.666667,26.666667
b,3.500000,35.000000


In [9]:
df.groupby(['key1', 'key2']).mean()

data1  data2
key1 key2              
a    one       3     30
     two       2     20
b    one       3     30
     two       4     40

In [10]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterating Over Groups

In [11]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2  data1  data2
0    a  one      1     10
1    a  two      2     20
4    a  one      5     50
b
  key1 key2  data1  data2
2    b  one      3     30
3    b  two      4     40


In [12]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2  data1  data2
0    a  one      1     10
4    a  one      5     50
('a', 'two')
  key1 key2  data1  data2
1    a  two      2     20
('b', 'one')
  key1 key2  data1  data2
2    b  one      3     30
('b', 'two')
  key1 key2  data1  data2
3    b  two      4     40


In [13]:
pieces = dict(list(df.groupby('key1')))
pieces

{'a':   key1 key2  data1  data2
 0    a  one      1     10
 1    a  two      2     20
 4    a  one      5     50,
 'b':   key1 key2  data1  data2
 2    b  one      3     30
 3    b  two      4     40}

In [14]:
df.dtypes

key1     object
key2     object
data1     int64
data2     int64
dtype: object

In [15]:
grouped = df.groupby(df.dtypes, axis=1)
for dtype, group in grouped:
    print(dtype)
    print(group)

int64
   data1  data2
0      1     10
1      2     20
2      3     30
3      4     40
4      5     50
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### Selecting a Column or Subset of Columns

```
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]
```
The code above is syntactic sugar for:
```
df['data1'].groupby(df['key1'])
df['data2'].groupby(df['key2'])
```

### Grouping with Dicts and Series

In [16]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values

people

,a,b,c,d,e
Joe,0.065172,0.049402,0.400802,-1.276665,-0.229348
Steve,-0.410832,-1.072244,-0.561248,0.054248,0.365091
Wes,2.086301,NaN,NaN,2.322939,0.450428
Jim,0.708291,-2.288891,0.129439,-0.588895,-0.585511
Travis,2.757474,2.493545,-1.534536,0.476205,-0.669102


In [17]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [18]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,-0.875863,-0.114773
Steve,-0.507000,-1.117986
Wes,2.322939,2.536729
Jim,-0.459455,-2.166111
Travis,-1.058331,4.581917


In [19]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [20]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### Grouping with functions

In [21]:
people.groupby(len).sum()

,a,b,c,d,e
3,2.859764,-2.239489,0.530242,0.457379,-0.364430
5,-0.410832,-1.072244,-0.561248,0.054248,0.365091
6,2.757474,2.493545,-1.534536,0.476205,-0.669102


In [22]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one  0.065172  0.049402  0.400802 -1.276665 -0.229348
  two  0.708291 -2.288891  0.129439 -0.588895 -0.585511
5 one -0.410832 -1.072244 -0.561248  0.054248  0.365091
6 two  2.757474  2.493545 -1.534536  0.476205 -0.669102

### Grouping by Index Levels

In [23]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'], [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)

hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.757914 -0.218456  1.468185 -0.366556 -1.485052
1     -0.188934  0.368837 -0.081553 -3.078487  0.621704
2      0.766046  1.511157  0.144577  0.314197  1.843052
3      1.126098  0.008638 -0.118690 -1.196712 -2.753709

In [24]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 Data Aggregation

Aggregation: any data transformation that produces scalar values from arrays.

In [25]:
df

,key1,key2,data1,data2
0,a,one,1,10
1,a,two,2,20
2,b,one,3,30
3,b,two,4,40
4,a,one,5,50


In [26]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    4.4
b    3.9
Name: data1, dtype: float64

In [27]:
# custom function is much slower than the optimized builtin functions
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,4,40
b,1,10


In [28]:
grouped.describe()

data1                                                data2             \
     count      mean       std  min   25%  50%   75%  max count       mean   
key1                                                                         
a      3.0  2.666667  2.081666  1.0  1.50  2.0  3.50  5.0   3.0  26.666667   
b      2.0  3.500000  0.707107  3.0  3.25  3.5  3.75  4.0   2.0  35.000000   

                                               
            std   min   25%   50%   75%   max  
key1                                           
a     20.816660  10.0  15.0  20.0  35.0  50.0  
b      7.071068  30.0  32.5  35.0  37.5  40.0

### Column-Wise and Multiple Function Application

In [29]:
tips = pd.read_csv('pydata-book/examples/tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [30]:
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [31]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [32]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [33]:
functions = ['count', 'mean', 'max']
result = grouped[['tip_pct', 'total_bill']].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [34]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [35]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped[['tip_pct', 'total_bill']].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [36]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [37]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Returning Aggregated Data Without Row Indexes

In [38]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## 10.3 Apply: General split-apply-combine

In [39]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [40]:
# top() is call on each row group and the results are glued using pandas.concat
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [41]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [42]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [43]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

### Suppressing the Group Keys

In [44]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### Quantile and Bucket Analysis

In [45]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
quantiles = pd.cut(frame.data1, 4)
quantiles[:10]

0    (-1.11, 0.374]
1    (-1.11, 0.374]
2    (-1.11, 0.374]
3    (-1.11, 0.374]
4    (-1.11, 0.374]
5    (-1.11, 0.374]
6    (0.374, 1.858]
7    (0.374, 1.858]
8    (-1.11, 0.374]
9    (-1.11, 0.374]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.6, -1.11] < (-1.11, 0.374] < (0.374, 1.858] < (1.858, 3.341]]

In [46]:
quantiles

0      (-1.11, 0.374]
1      (-1.11, 0.374]
2      (-1.11, 0.374]
3      (-1.11, 0.374]
4      (-1.11, 0.374]
            ...      
995    (-1.11, 0.374]
996     (-2.6, -1.11]
997    (-1.11, 0.374]
998    (0.374, 1.858]
999     (-2.6, -1.11]
Name: data1, Length: 1000, dtype: category
Categories (4, interval[float64]): [(-2.6, -1.11] < (-1.11, 0.374] < (0.374, 1.858] < (1.858, 3.341]]

In [47]:
frame['data1'][:10]

0   -0.119194
1   -0.986675
2    0.360057
3    0.334649
4    0.003175
5   -0.557615
6    0.711883
7    1.709787
8   -0.628009
9   -0.080570
Name: data1, dtype: float64

In [49]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}

grouped = frame.data2.groupby(quantiles)
grouped.apply(get_stats)

data1                
(-2.6, -1.11]   min       -2.752190
                max        2.470728
                count    117.000000
                mean      -0.107980
(-1.11, 0.374]  min       -2.726742
                max        2.831904
                count    511.000000
                mean      -0.051142
(0.374, 1.858]  min       -3.283748
                max        2.784075
                count    347.000000
                mean      -0.099233
(1.858, 3.341]  min       -1.999090
                max        1.558218
                count     25.000000
                mean      -0.123463
Name: data2, dtype: float64

In [50]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-2.6, -1.11]",-2.752190,2.470728,117.0,-0.107980
"(-1.11, 0.374]",-2.726742,2.831904,511.0,-0.051142
"(0.374, 1.858]",-3.283748,2.784075,347.0,-0.099233
"(1.858, 3.341]",-1.999090,1.558218,25.0,-0.123463


In [51]:
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.752190,2.052156,100.0,-0.116359
1,-2.416242,2.470728,100.0,-0.117536
2,-2.202709,2.831904,100.0,0.013683
3,-2.607502,2.165891,100.0,0.000787
4,-2.470132,1.943555,100.0,0.044399
5,-2.726742,2.744627,100.0,-0.077321
6,-2.675999,2.159885,100.0,-0.273442
7,-2.525409,2.784075,100.0,-0.032617
8,-3.283748,1.987028,100.0,-0.158403


### Example: Filling Missing Values with Group-Specific Values

In [52]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -1.691546
2         NaN
3    0.086356
4         NaN
5    1.497941
dtype: float64

In [53]:
s.fillna(s.mean())

0   -0.035750
1   -1.691546
2   -0.035750
3    0.086356
4   -0.035750
5    1.497941
dtype: float64

In [54]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data

Ohio         -1.210778
New York      0.834282
Vermont       0.226922
Florida       0.191945
Oregon       -0.780452
Nevada        0.350745
California   -0.537813
Idaho         1.402111
dtype: float64

In [55]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio         -1.210778
New York      0.834282
Vermont            NaN
Florida       0.191945
Oregon       -0.780452
Nevada             NaN
California   -0.537813
Idaho              NaN
dtype: float64

In [56]:
data.groupby(group_key).mean()

East   -0.061517
West   -0.659132
dtype: float64

In [57]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio         -1.210778
New York      0.834282
Vermont      -0.061517
Florida       0.191945
Oregon       -0.780452
Nevada       -0.659132
California   -0.537813
Idaho        -0.659132
dtype: float64

In [58]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio         -1.210778
New York      0.834282
Vermont       0.500000
Florida       0.191945
Oregon       -0.780452
Nevada       -1.000000
California   -0.537813
Idaho        -1.000000
dtype: float64

### Example: Random Sampling and Permutation

In [59]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
# (I thought J, K, and Q are 11, 12, and 13.)
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

In [60]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [61]:
def draw(deck, n=5):
    return deck.sample(n)

draw(deck)

2H      2
KD     10
3D      3
10H    10
10C    10
dtype: int64

In [62]:
get_suit = lambda card: card[-1] # last letter is suit
deck.groupby(get_suit).apply(draw, n=2)

C  4C      4
   JC     10
D  10D    10
   AD      1
H  2H      2
   4H      4
S  3S      3
   KS     10
dtype: int64

In [63]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

5C      5
10C    10
10D    10
3D      3
6H      6
9H      9
2S      2
KS     10
dtype: int64

### Example: Group Weighted Average and Correlation

In [64]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

,category,data,weights
0,a,1.385426,0.572746
1,a,-1.303332,0.879945
2,a,0.356289,0.698674
3,a,0.724124,0.042923
4,b,-0.629296,0.762417
5,b,-0.714002,0.450786
6,b,0.115828,0.608445
7,b,1.064485,0.477601


In [65]:
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

category
a   -0.033428
b   -0.096890
dtype: float64

In [66]:
close_px = pd.read_csv('pydata-book/examples/stock_px_2.csv', parse_dates=True, 
                       index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [67]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [68]:
spx_corr = lambda x: x.corrwith(x['SPX'])
rets = close_px.pct_change().dropna()
get_year = lambda x: x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [69]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### Example: Group-Wise Linear Regression

In [70]:
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    # OLS: Ordinary Least Square
    result = sm.OLS(Y, X).fit()
    return result.params

In [71]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


# 10.4 Pivot Tables and Cross-Tabulation

Pivot table

* is a data summarization tool
* aggregates a table of data by one or more keys, arranging the data in a rectangle with some of the group keys along the rows and some along the columns

In [72]:
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [73]:
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [74]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [75]:
# to include partial totals
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [76]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',
                 aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [77]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

### Cross-Tabulations: Crosstab

In [79]:
from io import StringIO
data = """\
Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep='\s+')

In [80]:
# cross tabulation aka contingency table, cross tabs
pd.crosstab(data.Nationality, data.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [81]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244